In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
import numpy as np

In [4]:
train_df = pd.read_csv('Resources/2019loans.csv')
test_df = pd.read_csv('Resources/2020Q1loans.csv')

In [5]:
#checking the data to validate non-numeric values of targets
print(train_df["target"].value_counts())
print(test_df["target"].value_counts())

low_risk     6090
high_risk    6090
Name: target, dtype: int64
high_risk    2351
low_risk     2351
Name: target, dtype: int64


In [6]:
# Convert categorical data to numeric and separate target feature for training data
# Drop the label to create the X data
train_X_step = train_df.drop('target', axis=1)
test_X_step = test_df.drop('target', axis=1)

# categorical conversion
train_X = pd.get_dummies(train_X_step)
test_X = pd.get_dummies(test_X_step)

#create the targets
train_Y = train_df["target"]
test_Y = test_df["target"]

In [7]:
# add missing dummy variables to testing set
for X in train_X.columns:
    if X not in test_X.columns:
        test_X[X]=0

### Model Consideration

Considering this data is pruned for the purposes of this data set, I would predict that the Logistic Regression would be a better fit. Random Forest would be probably better for the very large, unprocessed data set (I think?).

In [8]:
# Train the Logistic Regression model on the unscaled data and print the model score
classifier = LogisticRegression()

classifier.fit(train_X, train_Y)

print(f"Training Data Score: {classifier.score(train_X, train_Y)}")
print(f"Testing Data Score: {classifier.score(test_X, test_Y)}")

log_summary = pd.DataFrame([{"Model": "Non-Scaled LogisticRegression",
                            "Training Score":classifier.score(train_X, train_Y),
                            "Testing Score":classifier.score(test_X, test_Y),}])

log_summary

Training Data Score: 0.6532019704433497
Testing Data Score: 0.5082943428328371


C:\Users\andre\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Model,Training Score,Testing Score
0,Non-Scaled LogisticRegression,0.653202,0.508294


In [9]:
# Train a Random Forest Classifier model and print the model score
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(random_state=1, n_estimators=500)

forest.fit(train_X, train_Y)

print(f'Training Score: {forest.score(train_X, train_Y)}')
print(f'Testing Score: {forest.score(test_X, test_Y)}')

for_summary = pd.DataFrame([{"Model": "Non-Scaled Random Forest Classifier",
                            "Training Score":forest.score(train_X, train_Y),
                            "Testing Score":forest.score(test_X, test_Y),}])

Training Score: 1.0
Testing Score: 0.646958740961293


In [10]:
# Scale the data
scaler = StandardScaler()

scaler.fit(train_X)

train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [11]:
# Train the Logistic Regression model on the scaled data and print the model score
classifier = LogisticRegression()

classifier.fit(train_X_scaled, train_Y)

print(f"Training Data Score: {classifier.score(train_X_scaled, train_Y)}")
print(f"Testing Data Score: {classifier.score(test_X_scaled, test_Y)}")

log_scaled_summary = pd.DataFrame([{"Model": "Scaled LogisticRegression",
                            "Training Score":classifier.score(train_X_scaled, train_Y),
                            "Testing Score":classifier.score(test_X_scaled, test_Y),}])

log_scaled_summary

Training Data Score: 0.710919540229885
Testing Data Score: 0.7598894087622289


C:\Users\andre\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Model,Training Score,Testing Score
0,Scaled LogisticRegression,0.71092,0.759889


In [12]:
# Train a Random Forest Classifier model on the scaled data and print the model score

forest = RandomForestClassifier(random_state=1, n_estimators=500)

forest.fit(train_X_scaled, train_Y)

print(f'Training Score: {forest.score(train_X_scaled, train_Y)}')
print(f'Testing Score: {forest.score(test_X_scaled, test_Y)}')

for_scaled_summary = pd.DataFrame([{"Model": "Scaled Random Forest Classifier",
                            "Training Score":forest.score(train_X_scaled, train_Y),
                            "Testing Score":forest.score(test_X_scaled, test_Y),}])

for_scaled_summary

Training Score: 1.0
Testing Score: 0.6480221182475542


,Model,Training Score,Testing Score
0,Scaled Random Forest Classifier,1.0,0.648022


In [15]:
#putting all the values in one table for quick comparison
summary_df = pd.concat([log_summary,log_scaled_summary,for_summary,for_scaled_summary])
summary_df

,Model,Training Score,Testing Score
0,Non-Scaled LogisticRegression,0.653202,0.508294
0,Scaled LogisticRegression,0.710920,0.759889
0,Non-Scaled Random Forest Classifier,1.000000,0.646959
0,Scaled Random Forest Classifier,1.000000,0.648022
